In [1]:
import cv2 as cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def open_pictures(img_name):
    img_rgb = cv2.imread(img_name)
    img_gray = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
    return img_rgb, img_gray

In [3]:
def get_mask(img_name):
    img = cv2.imread(img_name)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    max_value = 255
    max_value_H = 130
    low_H = 80
    low_S = 0
    low_V = 0
    high_H = max_value_H
    high_S = max_value
    high_V = max_value
    # применяем фильтр, делаем бинаризацию
    mask = cv2.inRange(hsv, (low_H, low_S, low_V), (high_H, high_S, high_V))
    return ~mask

In [4]:
def preprocessing(img_gray, mask):
    blur = cv2.GaussianBlur(img_gray, (11, 11), 0)
    canny = cv2.Canny(blur, 0, 40, 5)
    
    #dilate
    dilated = cv2.dilate(canny - mask, (3, 3), iterations=1) 
    
    #close
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    closed = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
    return closed

In [5]:
def get_fig_contours(img):
    (cnt, hierarchy) = cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    im = np.zeros(img.shape)
    
    #отбираем контуры по площади и периметру 
    for c in cnt:
        peri = cv2.arcLength(c, True)
        if (5000 >cv2.contourArea(c)>1100) & (100 < peri < 1000):
            cv2.drawContours(im, c, -1, (255, 255, 255), 2)
    
    im = im.astype('uint8')       
    (cnt, hierarchy) = cv2.findContours(im, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_NONE)
    return cnt

In [6]:
def num_cards(img_name):
    rgb, gray = open_pictures(img_name)
    
    preproc = preprocessing(gray, get_mask(img_name))
    cnt = get_fig_contours(preproc)
    return len(cnt)

In [7]:
def draw_figures(cnt, shape):
    im = np.zeros(shape)
    for c in cnt:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if 100 < peri < 1000:
            cv2.drawContours(im, c, -1, (255, 255, 255))
    im = im.astype('uint8')
    (cnt, hierarchy) = cv2.findContours(im, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(im, cnt, -1, (255, 255, 255), thickness=cv2.FILLED)
    return im.astype('uint8')

In [8]:
def aprox_figures(img):
    dilated = cv2.dilate(img, (4, 4), iterations=2)
    kernel = np.ones((2, 2),np.uint8)
    eros = cv2.erode(dilated, kernel, iterations = 5)
    return eros

In [9]:
def draw_res(img_rgb, aprox, draw_cont=False):
    (cnt, hierarchy) = cv2.findContours(aprox, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_NONE)
    
    for c in cnt:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if draw_cont & (100 < peri < 1000):
            cv2.drawContours(img_rgb, c, -1, (0, 255, 0), 2)
            
        x, y, w, h = cv2.boundingRect(c)
        if 100 < peri < 1000:
            if cv2.isContourConvex(approx):
                cv2.putText(img_rgb, f'{len(approx)}C', org=(x, y+h//3), 
                            fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, 
                            color=(0, 0, 255),thickness=2)
            else:
                cv2.putText(img_rgb, f'{len(approx)}', org=(x, y+h//3), 
                            fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, 
                            color=(0, 0, 255),thickness=2)
    return img_rgb

In [10]:
def find_figures(img_name, draw_cont=False):
    rgb, gray = open_pictures(img_name)
    
    preproc = preprocessing(gray, get_mask(img_name))
    cnt = get_fig_contours(preproc)
    figures = draw_figures(cnt, gray.shape)
    pict = aprox_figures(figures)
    
    res = draw_res(rgb, pict, draw_cont=draw_cont)
    return res

In [29]:
img_name = "IMG_1.jpg"

In [30]:
num_cards(img_name)

16

In [31]:
res = find_figures(img_name)
cv2.imshow('contours', res)
#cv2.imwrite('IMG_4_res.jpg', res)
cv2.waitKey()

-1